<a href="https://colab.research.google.com/github/Amir-Fasil/Prompt_Vs_finetuning/blob/main/Few_shot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain transformers torch sentence-transformers langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.6

In [ ]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.chains import LLMChain
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from google.colab import userdata
from huggingface_hub import login
import os
import torch

**Initialize the Llama 2 model and tokenizer**

In [ ]:
hf_token = userdata.get('HF_TOKEN')
login(token=hf_token, add_to_git_credential=True)

In [ ]:
# Initialize the Llama 2 model and tokenizer
def load_llama2_model():
    model_name = "meta-llama/Llama-2-7b-chat-hf"

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Load model
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        torch_dtype=torch.float16,
    )

    # Create text generation pipeline
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=512,
        temperature=0.7,
        top_p=0.95,
        repetition_penalty=1.15
    )

    # Wrap in LangChain HuggingFacePipeline
    llm = HuggingFacePipeline(pipeline=pipe)

    return llm

llm = load_llama2_model()

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Device set to use cuda:0
<ipython-input-5-4eebcc4ede0b>:27: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


Few_shot_prompting

In [ ]:
few_shot_examples = [
    {
        "question": "What is the capital of France?",
        "answer": "The capital of France is Paris."
    },
    {
        "question": "What is the capital of Germany?",
        "answer": "The capital of Germany is Berlin."
    },
    {
        "question": "What is the capital of Japan?",
        "answer": "The capital of Japan is Tokyo."
    }
]

def create_few_shot_prompt():

      # This is the overall prompt structure

      example_template = """

      <s>[INST] <<SYS>>
      You are an expert in geography. Answer concisely.
      <</SYS>>
      Question: {question} [/INST]
      Answer: {answer}</s>
      """

      example_prompt = PromptTemplate(
          input_variables=["question", "answer"],
          template=example_template
      )

      return FewShotPromptTemplate(
          examples= few_shot_examples,
          example_prompt= example_prompt,
          prefix="<s>[INST] <<SYS>>\nAnswer the question based on the examples.\n<</SYS>>\n",
          suffix="Question: {input} [/INST]\nAnswer:",
          input_variables=["input"],
          example_separator="\n"
      )
prompt_template = create_few_shot_prompt()
prompt_template

FewShotPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, examples=[{'question': 'What is the capital of France?', 'answer': 'The capital of France is Paris.'}, {'question': 'What is the capital of Germany?', 'answer': 'The capital of Germany is Berlin.'}, {'question': 'What is the capital of Japan?', 'answer': 'The capital of Japan is Tokyo.'}], example_prompt=PromptTemplate(input_variables=['answer', 'question'], input_types={}, partial_variables={}, template='\n\n      <s>[INST] <<SYS>>\n      You are an expert in geography. Answer concisely.\n      <</SYS>>\n      Question: {question} [/INST]\n      Answer: {answer}</s>\n      '), suffix='Question: {input} [/INST]\nAnswer:', example_separator='\n', prefix='<s>[INST] <<SYS>>\nAnswer the question based on the examples.\n<</SYS>>\n')

**Wrappingg Lanchain Chain on to the model and the prompt**

In [ ]:
def create_zero_shot_chain(llm, prompt_template):

    return LLMChain(

        llm=llm,
        prompt=prompt_template,
        verbose=True  # Set to True to see the prompt and response details
    )

few_shot_chain = create_zero_shot_chain(llm, prompt_template)

In [ ]:
def main(few_shot_chain, few_shot_examples):


    # Test questions
    test_questions = [

        "What is the capital of Italy?",
        "Which team won Super Bowl LVIII in 2024?",
        "How do I bake a chocolate cake without eggs?"
    ]

    for question in test_questions:
        print(f"\nQuestion: {question}")
        response = few_shot_chain.run(input= question)
        print(f"Answer: {response}")



main(few_shot_chain, few_shot_examples)


Question: What is the capital of Italy?


> Entering new LLMChain chain...
Prompt after formatting:
<s>[INST] <<SYS>>
Answer the question based on the examples.
<</SYS>>



      <s>[INST] <<SYS>>
      You are an expert in geography. Answer concisely.
      <</SYS>>
      Question: What is the capital of France? [/INST]
      Answer: The capital of France is Paris.</s>
      


      <s>[INST] <<SYS>>
      You are an expert in geography. Answer concisely.
      <</SYS>>
      Question: What is the capital of Germany? [/INST]
      Answer: The capital of Germany is Berlin.</s>
      


      <s>[INST] <<SYS>>
      You are an expert in geography. Answer concisely.
      <</SYS>>
      Question: What is the capital of Japan? [/INST]
      Answer: The capital of Japan is Tokyo.</s>
      
Question: What is the capital of Italy? [/INST]
Answer:

> Finished chain.
Answer: <s>[INST] <<SYS>>
Answer the question based on the examples.
<</SYS>>



      <s>[INST] <<SYS>>
      You are an exp